# Instalação de bibliotecas no ambiente do Sage Maker

In [1]:
!pip install -U pip
!pip install sagemaker==2.15.3
!pip install boto3==1.16.2
!pip install pandas==1.0.1
!pip install numpy==1.18.1
!pip install matplotlib==3.3.3
!pip install scikit-learn==0.24.0

# Preparação de variáveis para o treinamento 

In [2]:
import boto3

sessao_boto = boto3.Session(region_name="us-east-1")

In [3]:
from sagemaker import get_execution_role

ROLE = get_execution_role()

print(f'Role de Execução: {ROLE}')

Role de Execução: arn:aws:iam::962752222089:role/service-role/AWSGlueServiceSageMakerNotebookRole-gdelt


In [4]:
import sagemaker

sessao_sagemaker = sagemaker.Session(boto_session=sessao_boto)

In [5]:
sessao_sagemaker

In [6]:
BUCKET ='sagemaker-alura-engenharia-machine-learning'

PREFIXO_DADOS = 'dados'

PREFIXO_MODELOS = 'modelos'

REGIAO = sessao_sagemaker.boto_region_name

SEMENTE = 42

# Carga de dados no S3 e Training Inputs

In [7]:
# Vamos colocar os caminhos em um dicionario
# apenas para manter a simplicidade e a consistencia
# dos caminhos. Em caso de mudança, essa será refletida
# em todos os caminhos.

dict_caminho_dados = {
    "treino": "dados/df_bytebank_treino.csv",
    "teste": "dados/df_bytebank_teste.csv",
    "validacao": "dados/df_bytebank_validacao.csv",    
}

In [8]:
# Upload dos dados no S3. Uma observação é que esse
# objeto do tipo "upload_data" ele mantem a referencia de
# metadados do S3 e pode ser usado como "canal" de treinamento#
# para qualquer estimador

bytebank_treino_caminho \
    = sessao_sagemaker.upload_data(
        path=dict_caminho_dados['treino'],
        bucket=BUCKET,
        key_prefix=f'{PREFIXO_DADOS}/treino')

bytebank_teste_caminho \
    = sessao_sagemaker.upload_data(
        path=dict_caminho_dados['teste'],
        bucket=BUCKET,
        key_prefix=f'{PREFIXO_DADOS}/teste')

bytebank_validacao_caminho \
    = sessao_sagemaker.upload_data(
        path=dict_caminho_dados['validacao'],
        bucket=BUCKET,
        key_prefix=f'{PREFIXO_DADOS}/validacao')

In [9]:
# Aqui são os canais que passaremos via training input para
# o estimador
s3_input_treino \
    = sagemaker.TrainingInput(s3_data=bytebank_treino_caminho,
                                          content_type='csv')

s3_input_teste \
    = sagemaker.TrainingInput(s3_data=bytebank_teste_caminho,
                                         content_type='csv')

# Estimador e construção do modelo

In [10]:
# Aqui chamamos o estimador do Scikit-Learn no Sage Maker
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    framework_version='0.20.0',
    py_version='py3',
    entry_point='script_customizado_sage_maker.py',
    code_location=f's3://{BUCKET}/{PREFIXO_MODELOS}',
    hyperparameters={'n_estimators': 100,'max_depth': 7},
    role=ROLE,
    instance_type='ml.c4.xlarge',
    output_path=f's3://{BUCKET}/{PREFIXO_MODELOS}',
    base_job_name='codigoCustomizadoEndpoint',
    sagemaker_session=sessao_sagemaker)

# Treinamento do modelo 

In [11]:
# Realizamos o ajuste do modelo invocando o nosso estimador
sklearn_estimator.fit({'train': s3_input_treino})

2021-02-15 22:14:03 Starting - Starting the training job...
2021-02-15 22:14:06 Starting - Launching requested ML instances......
2021-02-15 22:15:18 Starting - Preparing the instances for training......
2021-02-15 22:16:11 Downloading - Downloading input data...
2021-02-15 22:17:01 Training - Training image download completed. Training in progress..2021-02-15 22:17:01,885 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2021-02-15 22:17:01,887 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-15 22:17:01,899 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-02-15 22:17:02,110 botocore.utils INFO     IMDS ENDPOINT: http://169.254.169.254/
2021-02-15 22:17:02,263 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-15 22:17:02,277 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-15 22:17:02,289


2021-02-15 22:17:43 Uploading - Uploading generated training model
2021-02-15 22:17:43 Completed - Training job completed
Training seconds: 92
Billable seconds: 92


# Deploy do modelo no endpoint

In [12]:
# Deploy do modelo em um endpoint e aqui instanciamos o 
# serializador para indicarmos que vamos passar um 
# input do tipo CSV. 
from sagemaker.serializers import CSVSerializer
serializer = CSVSerializer()

predictor = sklearn_estimator.deploy(instance_type='ml.c4.xlarge',
                                     initial_instance_count=1,
                                     endpoint_name='codigoCustomizadoEndpoint',
                                     serializer=serializer
                                    )

---------------!

# Chamada do modelo no endpoint

In [13]:
# Vamos usar os dados que temos na base de validacão
# para realizar algumas inferencias
import pandas as pd

df_validacao = pd.read_csv(dict_caminho_dados['validacao'])
X_validacao = df_validacao.iloc[:,1:]
y_validacao = df_validacao.iloc[:,0]

In [14]:
df_validacao.head(5)

,INADIMPLENTE,CREDITO_CONCEDIDO,GENERO,ESCOLARIDADE,ESTADO_CIVIL,IDADE,ATRASO_PARCELA_M_0,ATRASO_PARCELA_M_2,ATRASO_PARCELA_M_3,ATRASO_PARCELA_M_4,...,SALDO_PAGO_PARCELA_3,SALDO_PAGO_PARCELA_4,SALDO_PAGO_PARCELA_5,SALDO_PAGO_PARCELA_6,VALOR_PAGAMENTO_ANTERIOR_1,VALOR_PAGAMENTO_ANTERIOR_2,VALOR_PAGAMENTO_ANTERIOR_3,VALOR_PAGAMENTO_ANTERIOR_4,VALOR_PAGAMENTO_ANTERIOR_5,VALOR_PAGAMENTO_ANTERIOR_6
0,0,10000,1,2,2,24,2,0,0,0,...,4810,7546,9260,10000,3000,2000,3000,2000,1000,0
1,0,360000,1,2,1,45,2,2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
2,0,280000,2,1,2,28,-1,-1,-1,0,...,10002,13476,23572,14603,3658,10020,10000,11000,14603,3819
3,1,52743,1,1,2,47,0,0,0,0,...,13507,11462,8191,9460,1091,10505,1603,658,3783,1790
4,1,243160,1,2,2,40,2,0,0,0,...,241034,240190,238301,238645,9454,9366,12440,9034,6587,11570


In [15]:
X_validacao = df_validacao.iloc[2:3, 1:]
X_validacao

,CREDITO_CONCEDIDO,GENERO,ESCOLARIDADE,ESTADO_CIVIL,IDADE,ATRASO_PARCELA_M_0,ATRASO_PARCELA_M_2,ATRASO_PARCELA_M_3,ATRASO_PARCELA_M_4,ATRASO_PARCELA_M_5,...,SALDO_PAGO_PARCELA_3,SALDO_PAGO_PARCELA_4,SALDO_PAGO_PARCELA_5,SALDO_PAGO_PARCELA_6,VALOR_PAGAMENTO_ANTERIOR_1,VALOR_PAGAMENTO_ANTERIOR_2,VALOR_PAGAMENTO_ANTERIOR_3,VALOR_PAGAMENTO_ANTERIOR_4,VALOR_PAGAMENTO_ANTERIOR_5,VALOR_PAGAMENTO_ANTERIOR_6
2,280000,2,1,2,28,-1,-1,-1,0,0,...,10002,13476,23572,14603,3658,10020,10000,11000,14603,3819


In [16]:
# Input que iremos passar para o nosso preditor
# aqui no momento do treinamento
csv_text = '140000,2,2,1,37,0,0,0,0,0,0,58081,51013,54343,27537,9751,12569,5000,5000,5000,3000,3000,5000'

print(predictor.predict(csv_text))

[array(['0'], dtype=object) array([0.751342, 0.248658])]


# Podemos realizar a chamada do nosso endpoint atraves de do objeto do tipo "Predictor"

In [17]:
import boto3
import json
import numpy as np
from sagemaker.serializers import CSVSerializer
from sagemaker.predictor import Predictor

serializer = CSVSerializer()

predictor = Predictor(
    endpoint_name='codigoCustomizadoEndpoint',
    sagemaker_session=sessao_sagemaker,
    serializer=serializer)

print(predictor.predict(csv_text))

b'"[[{\\"features\\":[\\"0\\"]},{\\"features\\":[0.7513419953,0.2486580047]}]]"'


# Podemos realizar a chamada do nosso endpoint usando o "invoke_endpoint"

In [18]:
import sagemaker

sessao_sagemaker = sagemaker.Session(boto_session=sessao_boto)

runtime = boto3.Session().client('sagemaker-runtime')
 
csv_text = '140000,2,2,1,37,0,0,0,0,0,0,58081,51013,54343,27537,9751,12569,5000,5000,5000,3000,3000,5000'

nome_endpoint='codigoCustomizadoEndpoint'

response = runtime.invoke_endpoint(EndpointName=nome_endpoint,
                                   ContentType='text/csv',
                                   Body=csv_text)

result = json.loads(response['Body'].read().decode())

result

'[[{"features":["0"]},{"features":[0.7513419953,0.2486580047]}]]'

# Podemos realizar a chamada do nosso endpoint usando o Boto3

In [19]:
import boto3

runtime \
    = boto3.Session().client(
        service_name='sagemaker-runtime',
        region_name="us-east-1")

payload \
    = '140000,2,2,1,37,0,0,0,0,0,0,58081,51013,54343,27537,9751,12569,5000,5000,5000,3000,3000,5000'

response \
    = runtime.invoke_endpoint(
        EndpointName=nome_endpoint,
        ContentType='text/csv',
        Body=payload)

predicao = (response['Body'].read().decode('UTF-8'))
print(f'Crédito será inadimplente?: {predicao}')

Crédito será inadimplente?: "[[{\"features\":[\"0\"]},{\"features\":[0.7513419953,0.2486580047]}]]"


In [20]:
# Deletar o endpoint
predictor.delete_endpoint()